In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-141226
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-141226


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name='udacity-a'

compute_config= AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)

aml_compute=ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import normal,uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    { 
      'C':choice(0.05,0.1,1),
      'max_iter':choice(20,50,100,200)
    }    
) 

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.05)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
config =ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=aml_compute
                     )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=100,
                                     max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr=exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
import joblib
# Get your best run and save the model from that run.

best_run=hdr.get_best_run_by_primary_metric()
best_run.get_details()

#best_model=LogisticRegression().fit(x_train,y_train)
#joblib.dump(best_model,'best_hdrvive_model.pkl')

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset = Dataset.Tabular.from_delimited_files(path=web_path)
ds = dataset.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classifaction',
    primary_metric='Accuracy',
    training_data=dataset,
    label_column_name=,
    n_cross_validations=2)

In [ ]:
# Submit your automl run

automl_run=exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best,fitted = automl_run.get_output()
print(best)

joblib.dump(fitted,'best_automl_model.pkl')